In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import os
import scipy.stats as stats
import random
from numpy import polyfit, poly1d

In [2]:
pa_year_neuro = pd.read_csv('PaperId_Year_chemistry.csv')
# citing_cited = pd.read_csv('citing_cited_chemistry.csv')
# neuro_er_mag = pd.read_csv('chemistry_er_mag.csv')
pa_au = pd.read_csv('paper_author_chemistry.csv')  #mag中原始的 paper author 对应
pa_au_matched = pd.read_csv('PaperId_PID_chemistry.csv')
neuro = pd.read_csv('chemistry_1206.csv')

In [3]:
pa_au_matched.head(2)

,PaperId,PID
0,524.0,85005
1,9093269.0,85005


In [43]:
# pa_au.shape

In [28]:
AuthorCount = dict(Counter(pa_au.PaperId))
#删除作者数大于25的文章
tp0 = np.array(list(AuthorCount.keys()))
tp1 = np.array(list(AuthorCount.values()))
t0 = tp0[tp1<=25]
# t1 = tp1[tp1<=25]
# TP = {}
# for i in range(len(t0)):
#     TP[t0[i]] = t1[i]    
pa_au = pa_au[pa_au.PaperId.isin(t0)]

In [42]:
# pa_au.shape

In [33]:
neuro_mentee = neuro.MenteeID[neuro.StopYear<2000]
neuro_mentor = neuro.MentorID[neuro.StopYear<2000]
neuro_er = list(pd.concat([neuro_mentee,neuro_mentor]).drop_duplicates())
print(len(neuro_er))

49248


In [39]:
# print(pa_au.head(1))
# print(pa_au_matched.head(1))                   

In [38]:
with open('Citations.pickle','rb')as f:
    Citations = pickle.load(f)

In [40]:
# pa_year_neuro.head(1)

In [41]:
# pa_year_neuro.Year[pa_year_neuro.PaperId==1512635943]

In [ ]:
pub_Info = {}
count = 0
for k in neuro_er[:25000]:
    print(count)
    pas_k = pa_au_matched.PaperId[pa_au_matched.PID==k]   # all papers of author k in MAG  1
#     print('pas_k:',pas_k)
    year_pas_k = np.array(pa_year_neuro.Year[pa_year_neuro.PaperId.isin(pas_k)]) # year of papers abovel
#     print('year_pas_k:',year_pas_k)
    if len(year_pas_k)>0:
        the_first_year = min(year_pas_k) # 2
        the_last_year = max(year_pas_k)
        career_len = the_last_year - the_first_year + 1 # 3 
        yearly_productivity = len(pas_k)/career_len # 4
        # 5
        tp = year_pas_k - the_first_year
#         print('tp:',tp)
        py_pas_k = np.array(pa_year_neuro.PaperId[pa_year_neuro.PaperId.isin(pas_k)]) # papers having Year
#         print('py_pas_k:',py_pas_k)
        theFirst5Year_pubs = py_pas_k[tp<=5]  # 5 

        # citation
    #     tc = citing_cited.cited[citing_cited.cited.isin(pas_k)]
    #     total_c = len(tc)                # 6 
        total_c = 0
        for pk in pas_k:
            try:
                tpc = Citations[pk]
            except:
                tpc = 0
            total_c+=tpc

        yearly_c = total_c/career_len    # 7
        # 8 
    #     tc5 = citing_cited.cited[citing_cited.cited.isin(theFirst5Year_pubs)]
        tc5 = 0
        for pk in theFirst5Year_pubs:
            try:
                tpc = Citations[pk]
            except:
                tpc = 0
            tc5+=tpc

        theFirst5Year_c = tc5      # 8
    
        #coauthor
        coau = pa_au.AuthorID[pa_au.PaperId.isin(pas_k)].drop_duplicates() #不重复的合作者  #9
        yearly_coau = len(coau)/career_len  # 10
        theFirst5Year_coau = pa_au.AuthorID[pa_au.PaperId.isin(theFirst5Year_pubs)].drop_duplicates() # 11

        #受训期间的导师发表情况
        traintion = {} #循环的是每个学生和其受训时间   # 11
        year_train = np.array(neuro[['MenteeID','StopYear']][neuro.MentorID==k].drop_duplicates())
        if len(year_train)>0:
            for ii in range(len(year_train)):
                year_pas_k1 = pd.DataFrame(year_pas_k)
                py_pas_k1 = pd.DataFrame(py_pas_k)

                tx = year_train[ii]
                yy = tx[1]  #上面学生受训的时间
                yt = py_pas_k1[0][(year_pas_k1[0]>=yy-3)&(year_pas_k1[0]<yy+3)]  #受训时期前后导师总共6年内的文章 11

                #找在期间学生与导师的合作情况
                ym = tx[0]  #具体受训的学生
                coauthered = np.array(pa_au_matched.PID[pa_au_matched.PaperId.isin(yt)])
                cn = len(coauthered[coauthered==ym])   # 12 受训期间该学生与导师的合作文章数量

                traintion[ym] = (len(yt),cn)           #key是受训的每个学生     
        else:
            traintion = {}
    
        pub_Info[k] = [len(pas_k),the_first_year,career_len,yearly_productivity,len(theFirst5Year_pubs),
                       total_c,yearly_c,theFirst5Year_c,len(coau),yearly_coau,len(theFirst5Year_coau),
                      traintion]
    else:
        pub_Info[k] = []
    count+=1
# 0.总发文量
# 1.出道年
# 2.生涯长度
# 3.年均文章数
# 4.前5年的发文量
# 5.总引用
# 6.年均引用
# 7.前五年内的总引用
# 8.总合作者数量
# 9.年均合作者数量
# 10.前5年的合作者数量
# 11.训练期间的文章发表与合作情况 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#save data
# with open('pub_Info0_25000.pickle','wb') as f:
#     pickle.dump(pub_Info,f)

In [2]:
# with open('pub_Info0_2500.pickle','rb') as f:
#     pub_Info = pickle.load(f)

In [3]:
pub_Info

{18761: [276,
  1986.0,
  36.0,
  7.666666666666667,
  19,
  15879,
  441.0833333333333,
  745,
  583,
  16.194444444444443,
  25,
  {18760: (68, 0),
   20259: (0, 0),
   37982: (0, 0),
   10786: (0, 0),
   54383: (41, 8),
   49078: (0, 0),
   4166: (19, 4),
   119255: (50, 0),
   663417: (0, 0),
   679452: (0, 0),
   780205: (39, 0)}],
 53604: [91,
  1995.0,
  27.0,
  3.3703703703703702,
  3,
  5466,
  202.44444444444446,
  165,
  204,
  7.555555555555555,
  4,
  {}],
 20: [60,
  1985.0,
  37.0,
  1.6216216216216217,
  6,
  5263,
  142.24324324324326,
  327,
  87,
  2.3513513513513513,
  8,
  {390: (0, 0),
   566: (0, 0),
   1260: (11, 0),
   4558: (11, 0),
   8871: (0, 0),
   8872: (16, 0),
   2397: (0, 0),
   15295: (12, 0),
   18688: (17, 0),
   55454: (11, 2),
   56612: (0, 0),
   102419: (6, 0),
   102420: (19, 0),
   688612: (0, 0),
   740458: (8, 0)}],
 12561: [134,
  1972.0,
  39.0,
  3.4358974358974357,
  11,
  3837,
  98.38461538461539,
  305,
  242,
  6.205128205128205,
  2